In [ ]:
# ---------------------------------------------------------
# phone_predictive_text.py
# Predictive Text Generator with Mobile Phone Style UI
# Author: Umme Kulsum
# Internship: Python Programming Internship with Algonive
# ---------------------------------------------------------
# Run: python phone_predictive_text.py   (or run in Jupyter)
# No external libraries required

import tkinter as tk
from tkinter import font
from collections import defaultdict, Counter
import re

# ---------- N-GRAM MODEL ----------
class NgramPredictor:
    def __init__(self, n=3):
        self.n = n
        self.model = defaultdict(Counter)
        self.vocab = set()

    def tokenize(self, text):
        return re.findall(r"[a-zA-Z']+", text.lower())

    def train(self, text):
        tokens = self.tokenize(text)
        self.vocab.update(tokens)
        if len(tokens) < self.n: return
        window = self.n
        for i in range(len(tokens) - window + 1):
            context = tuple(tokens[i:i+window-1])
            nxt = tokens[i+window-1]
            self.model[context][nxt] += 1

    def suggest(self, text, k=3):
        tokens = self.tokenize(text)
        if not tokens:
            commons = Counter()
            for ctx, nxts in self.model.items():
                commons.update(nxts)
            return [w for w,_ in commons.most_common(k)]
        for ctx_len in range(min(self.n-1, len(tokens)), 0, -1):
            context = tuple(tokens[-ctx_len:])
            candidates = Counter()
            for ctx, nxts in self.model.items():
                if ctx[-ctx_len:] == context:
                    candidates.update(nxts)
            if candidates:
                return [w for w,_ in candidates.most_common(k)]
        commons = Counter()
        for nxts in self.model.values():
            commons.update(nxts)
        return [w for w,_ in commons.most_common(k)]

    def complete_with(self, text, word, add_space=True):
        if not text.strip():
            return word
        sep = " " if add_space and not text.endswith(" ") else ""
        return text + sep + word

# ---------- TRAINING CORPUS ----------
TRAIN_TEXT = """
Artificial intelligence and natural language processing power modern predictive text.
Predictive typing learns patterns from previous words to suggest the next word.
Python is widely used for data science and machine learning projects.
Machine learning automates analytics and enables smart applications.
Great user experience needs speed, accuracy and intuitive design.
"""

# ---------- PHONE UI HELPERS ----------
def round_rect(canvas, x1, y1, x2, y2, r=24, **kw):
    points = [
        x1+r, y1,
        x2-r, y1,
        x2, y1,
        x2, y1+r,
        x2, y2-r,
        x2, y2,
        x2-r, y2,
        x1+r, y2,
        x1, y2,
        x1, y2-r,
        x1, y1+r,
        x1, y1,
    ]
    return canvas.create_polygon(points, **kw, smooth=True)

class PhoneFrame(tk.Tk):
    def __init__(self, title="Predictive Text - Umme Kulsum"):
        super().__init__()
        self.title(title)
        self.configure(bg="#e9eef6")
        self.resizable(False, False)

        # Phone dimensions
        self.phone_w, self.phone_h = 420, 820
        self.canvas = tk.Canvas(self, width=self.phone_w+40, height=self.phone_h+40,
                                highlightthickness=0, bg="#e9eef6")
        self.canvas.pack(padx=10, pady=10)

        # Phone body
        self.body = round_rect(self.canvas, 20, 20, 20+self.phone_w, 20+self.phone_h,
                               r=36, fill="#111315", outline="")
        # Screen
        self.screen = round_rect(self.canvas, 34, 64, 34+self.phone_w-28,
                                 64+self.phone_h-90, r=28, fill="#f8fafc", outline="")
        # Notch
        self.canvas.create_oval(200, 44, 280, 68, fill="#111315", outline="#111315")

        # Embed a real frame as "screen content"
        self.screen_frame = tk.Frame(self.canvas, bg="#f8fafc")
        self.screen_window = self.canvas.create_window(34+6, 64+6, anchor="nw",
                                                       width=self.phone_w-40,
                                                       height=self.phone_h-102,
                                                       window=self.screen_frame)

# ---------- APP ----------
class MobilePredictiveApp:
    def __init__(self, root):
        self.root = root
        self.predictor = NgramPredictor(n=3)
        self.predictor.train(TRAIN_TEXT)
        self._build_ui()

    def _build_ui(self):
        sf = self.root.screen_frame

        # Fonts
        self.hfont = font.Font(family="Helvetica", size=14, weight="bold")
        self.tfont = font.Font(family="Helvetica", size=12)
        self.kfont = font.Font(family="Helvetica", size=12, weight="bold")

        # Header
        header = tk.Frame(sf, bg="#0ea5e9")
        header.pack(fill="x")
        tk.Label(header, text="Predictive Text - Umme Kulsum",
                 fg="white", bg="#0ea5e9", font=self.hfont, pady=10).pack()

        # Text area
        self.text = tk.Text(sf, height=10, wrap="word", bd=0, relief="flat",
                            bg="#ffffff", fg="#0f172a", padx=12, pady=12)
        self.text.pack(fill="both", expand=True, padx=12, pady=(12,8))
        self.text.insert("1.0", "Type here…")

        # Suggestion chips
        self.chip_bar = tk.Frame(sf, bg="#f8fafc")
        self.chip_bar.pack(fill="x", padx=12, pady=(0,8))
        self.chips = []
        for _ in range(3):
            btn = tk.Button(self.chip_bar, text="—", command=lambda w="": self._apply_chip(w),
                            bg="#e2e8f0", fg="#0f172a", bd=0, padx=12, pady=6,
                            font=self.tfont, relief="flat", activebackground="#cbd5e1")
            btn.pack(side="left", padx=6)
            self.chips.append(btn)

        # Generate button
        self.gen_btn = tk.Button(sf, text="Suggest Next Words", command=self.refresh_suggestions,
                                 bg="#0ea5e9", fg="white", bd=0, padx=12, pady=10,
                                 font=self.tfont, relief="flat", activebackground="#0284c7")
        self.gen_btn.pack(fill="x", padx=12, pady=(0,8))

        # On-screen keyboard
        self._build_keyboard(sf)

        # Bindings
        self.text.bind("<KeyRelease>", lambda e: self.refresh_suggestions())

        # First suggestion
        self.refresh_suggestions()

    def _apply_chip(self, w):
        if not w or w == "—": return
        content = self.text.get("1.0", "end-1c")
        new_text = self.predictor.complete_with(content, w)
        self.text.delete("1.0", "end")
        self.text.insert("1.0", new_text)
        self.refresh_suggestions()

    def refresh_suggestions(self):
        content = self.text.get("1.0", "end-1c")
        suggs = self.predictor.suggest(content, k=3)
        for i, btn in enumerate(self.chips):
            btn.config(text=suggs[i] if i < len(suggs) else "—",
                       command=lambda w=(suggs[i] if i < len(suggs) else ""): self._apply_chip(w))

    def _build_keyboard(self, parent):
        kb = tk.Frame(parent, bg="#f1f5f9")
        kb.pack(fill="x", padx=8, pady=(0,8))

        rows = [
            list("qwertyuiop"),
            list("asdfghjkl"),
            ["⇧"] + list("zxcvbnm") + ["⌫"],
            ["123", "space", "enter"]
        ]

        def insert_char(ch):
            if ch == "space":
                self.text.insert("insert", " ")
            elif ch == "enter":
                self.text.insert("insert", "\n")
            elif ch == "⌫":
                self.text.delete("insert-1c")
            elif ch == "⇧":
                self.text.insert("insert", "")
            elif ch == "123":
                self._popup_digits()
            else:
                self.text.insert("insert", ch)
            self.refresh_suggestions()

        for r in rows:
            rowf = tk.Frame(kb, bg="#f1f5f9")
            rowf.pack(pady=3)
            for ch in r:
                label = " " * (2 if len(ch)==1 else 1) + ch + " " * (2 if len(ch)==1 else 1)
                b = tk.Button(rowf, text=label, bd=0, bg="#e2e8f0", fg="#0f172a", font=self.kfont,
                              activebackground="#cbd5e1", relief="flat",
                              command=lambda c=ch: insert_char(c))
                b.pack(side="left", padx=3)

    def _popup_digits(self):
        top = tk.Toplevel(self.root)
        top.title("Digits")
        top.configure(bg="#f8fafc")
        for i in range(10):
            tk.Button(top, text=str(i), width=4, bd=0, bg="#e2e8f0",
                      activebackground="#cbd5e1",
                      command=lambda c=str(i): (self.text.insert("insert", c),
                                                top.destroy(), self.refresh_suggestions())
                      ).grid(row=i//5, column=i%5, padx=4, pady=4)

# ---------- MAIN ----------
if __name__ == "__main__":
    app = PhoneFrame("Predictive Text - Umme Kulsum")
    MobilePredictiveApp(app)
    app.mainloop()